In [1]:
import numpy as np
import geopandas
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
import rasterio
from rasterio.plot import show

In [133]:
path = Path(r'C:\Users\jldz9\OneDrive - University of Missouri\MST\2022\2022SS\ArcGIS\ttem_specs\shapefile\Usgs water well')
df = pd.read_csv(path.joinpath('ParowanTable6.csv'))
gdf = gpd.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.long, df.lat))
gdf = gdf.set_crs(epsg=4326)
gdf = gdf.to_crs(epsg='32612')

xmin = gdf.geometry.x.min()
xmax = gdf.geometry.x.max()
ymin = gdf.geometry.y.min()
ymax = gdf.geometry.y.max()

x = np.arange(xmin-5000,xmax+5000,5000)
y = np.arange(ymin-5000,ymax+5000,5000)
gx,gy= np.meshgrid(x,y)
gz = np.full((y.shape[0], x.shape[0]),np.nan)

In [127]:
x.shape

(26,)

In [126]:
gy.shape

(31, 26)

In [125]:
gx.shape

(31, 26)

In [124]:
gz.shape

(31, 26)

In [134]:
xres=x[1]-x[0]
yres=y[1]-y[0]
xres

5000.0

In [135]:
for i in range(x.shape[0]):
    for j in range(y.shape[0]):
        tmp = gdf[(gdf.geometry.x<x[i]+xres/2)&(gdf.geometry.x>x[i]-xres/2)&(gdf.geometry.y<y[j]+yres/2)&(gdf.geometry.y>y[j]-yres/2)]
        gz[j,i]=tmp['Residue mg/L'].mean()

In [130]:
gz

array([[  nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan],
       [  nan, 333. ,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan],
       [  nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan,   nan, 279. ,   nan,   nan,   nan,
          nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan],
       [  nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan],
       [  nan,   nan,   nan, 295. ,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
          nan,   nan,   nan,  

In [136]:
transform=rasterio.Affine.translation(x[0]-xres/2,y[0]-yres/2)*rasterio.Affine.scale(xres,yres)
with rasterio.open(
    path.joinpath('parowantds.tif'),
    'w',
    driver='GTiff',
    height=gz.shape[0],
    width=gz.shape[1],
    count=1,
    dtype=gz.dtype,
    crs=rasterio.CRS.from_epsg(32612),
    transform=transform,
) as dst:
    dst.write(gz,1)

In [114]:
xx = np.linspace(-4.0, 4.0, 240)
yy = np.linspace(-3.0, 3.0, 180)

In [119]:
xx[1]-xx[0]


0.03347280334728042

In [118]:
yy[1]-yy[0]

0.03351955307262555